In [106]:
import requests
import json
import re
from pprint import pprint
import pandas as pd


def get_holdings(url,keys):
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:83.0) Gecko/20100101 Firefox/83.0"
    }
    # Initialize an empty list to store the extracted data
    holdings_data = []
    with requests.Session() as req:
        req.headers.update(headers)
        for key in keys:
            r = req.get(url.format(key))
            for line in r.text.splitlines():
                if not line.startswith('etf_holdings.formatted_data'):
                    continue
                data = json.loads(line[30:-1])
                for holding in data:
                    goal = re.search(r'etf/([^"]*)', holding[1])
                    if goal:
                        # Append relevant data to the list
                        holdings_data.append([goal.group(1), *holding[2:5]])
                break
    return holdings_data

In [107]:
schd_holdings = get_holdings("https://www.zacks.com/funds/etf/{}/holding",['schd'])
columns = ['symbol', 'shares', 'weight', '52wk_change']
df_holdings = pd.DataFrame(schd_holdings, columns=columns)
df_holdings

,symbol,shares,weight,52wk_change
0,VZ,"57,891,834",4.54,5.46
1,AMGN,"7,946,436",4.38,17.82
2,AVGO,"2,268,487",4.29,112.21
3,KO,"33,663,538",4.02,0.13
4,ABBV,"13,589,325",3.96,14.80
...,...,...,...,...
96,ETD,"369,919",0.02,12.88
97,EBF,"414,152",0.02,-10.80
98,HAFC,"488,760",0.02,-34.22
99,CPF,"439,861",0.02,-13.54


In [108]:
import numpy as np
vector_holdings = df_holdings.loc[:,['symbol','weight']]
vector_holdings['weight_trading212']=0.01
vector_schd = np.array(vector_holdings['weight'].astype(float))
initial_guess = np.array(vector_holdings['weight_trading212'].astype(float))

In [109]:
vector_holdings['symbol']= vector_holdings['symbol']+'_US_EQ'
vector_holdings

,symbol,weight,weight_trading212
0,VZ_US_EQ,4.54,0.01
1,AMGN_US_EQ,4.38,0.01
2,AVGO_US_EQ,4.29,0.01
3,KO_US_EQ,4.02,0.01
4,ABBV_US_EQ,3.96,0.01
...,...,...,...
96,ETD_US_EQ,0.02,0.01
97,EBF_US_EQ,0.02,0.01
98,HAFC_US_EQ,0.02,0.01
99,CPF_US_EQ,0.02,0.01


In [110]:
from scipy.spatial import distance
euclidean_distance = distance.euclidean(vector_holdings['weight'].astype(float), vector_holdings['weight_trading212'].astype(float))
euclidean_distance

16.726601567562973

In [111]:
vector_holdings = vector_holdings[vector_holdings['symbol'] != 'PARA_US_EQ']
vector_holdings = vector_holdings[vector_holdings['symbol'] != 'SNV_US_EQ']
vector_holdings = vector_holdings[vector_holdings['symbol'] != 'LAZ_US_EQ']
vector_holdings = vector_holdings[vector_holdings['symbol'] != 'CWENA_US_EQ']
vector_holdings = vector_holdings[vector_holdings['symbol'] != 'PFC_US_EQ']
vector_holdings = vector_holdings[vector_holdings['symbol'] != 'ETD_US_EQ']

In [112]:
vector_holdings.loc[len(vector_holdings.index)] = ['TREX_US_EQ', '0.05', 0.06] 

In [113]:
vector_holdings['weight_trading212'].sum()

1.0

In [114]:
json_data = vector_holdings.set_index('symbol').to_dict()['weight_trading212']

In [115]:
sum(json_data.values())

1.0000000000000007

In [116]:
'''
with open('schd_holdings212.json', 'w') as json_file:
    json.dump(json_data, json_file, indent=4)
'''

"\nwith open('schd_holdings212.json', 'w') as json_file:\n    json.dump(json_data, json_file, indent=4)\n"

In [117]:
json_data = vector_holdings.set_index('symbol').to_dict()['weight_trading212']
import requests

url = "https://live.trading212.com/api/v0/equity/pies"

payload = {
  "dividendCashAction": "REINVEST",
  "endDate": "2019-08-24T14:15:22Z",
  "goal": 0,
  "icon": "Home",
  "instrumentShares": filtered_data,
  "name": "SCHD replication"
}

headers = {
  "Content-Type": "application/json",
  "Authorization": "yuour key""
}

response = requests.post(url, json=payload, headers=headers)

data = response.json()
print(data)

{'code': 'InternalError'}


In [118]:
json_data

{'VZ_US_EQ': 0.01,
 'AMGN_US_EQ': 0.01,
 'AVGO_US_EQ': 0.01,
 'KO_US_EQ': 0.01,
 'ABBV_US_EQ': 0.01,
 'PEP_US_EQ': 0.01,
 'MRK_US_EQ': 0.01,
 'HD_US_EQ': 0.01,
 'BLK_US_EQ': 0.01,
 'TXN_US_EQ': 0.01,
 'UPS_US_EQ': 0.01,
 'PFE_US_EQ': 0.01,
 'CVX_US_EQ': 0.01,
 'CSCO_US_EQ': 0.01,
 'LMT_US_EQ': 0.01,
 'ADP_US_EQ': 0.01,
 'BX_US_EQ': 0.01,
 'MO_US_EQ': 0.01,
 'EOG_US_EQ': 0.01,
 'ITW_US_EQ': 0.01,
 'MMM_US_EQ': 0.01,
 'USB_US_EQ': 0.01,
 'VLO_US_EQ': 0.01,
 'KMB_US_EQ': 0.01,
 'F_US_EQ': 0.01,
 'PAYX_US_EQ': 0.01,
 'ALL_US_EQ': 0.01,
 'FAST_US_EQ': 0.01,
 'NEM_US_EQ': 0.01,
 'OKE_US_EQ': 0.01,
 'LYB_US_EQ': 0.01,
 'TROW_US_EQ': 0.01,
 'MTB_US_EQ': 0.01,
 'CTRA_US_EQ': 0.01,
 'FITB_US_EQ': 0.01,
 'DRI_US_EQ': 0.01,
 'NTRS_US_EQ': 0.01,
 'HBAN_US_EQ': 0.01,
 'RF_US_EQ': 0.01,
 'PKG_US_EQ': 0.01,
 'SNA_US_EQ': 0.01,
 'AMCR_US_EQ': 0.01,
 'BBY_US_EQ': 0.01,
 'K_US_EQ': 0.01,
 'TSN_US_EQ': 0.01,
 'IP_US_EQ': 0.01,
 'WSO_US_EQ': 0.01,
 'WSM_US_EQ': 0.01,
 'FNF_US_EQ': 0.01,
 'IPG_US_EQ': 0.01,